# SAS 해커톤에 쓰일 dummy data maker

### 기본세팅

In [48]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=10)
from faker import Faker
fake_ko = Faker('ko_KR')

import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict

In [2]:
# 데이터는 총 num_samples 개
## 수급자
num_samples = 100000
## 종사자

## 기관


## 수급자 정보

### L번호 생성

In [3]:
# l번호 생성
def generate_longtermcare_id():
    prefix = 'L'
    year = str(random.randint(8, 24)).zfill(2)  # 2자리로 만들기 위해 zfill 사용
    middle = '00'
    random_number = str(random.randint(0, 99999)).zfill(5)  # 6자리로 만들기 위해 zfill 사용
    
    # 최종 인정관리번호 생성
    longtermcare_id = f"{prefix}{year}{middle}{random_number}"    
    return longtermcare_id

In [4]:
# l번호 생성 후 저장
print(f'sample: {generate_longtermcare_id()}')
longtermcare_id = np.array([generate_longtermcare_id() for _ in range(num_samples)])
longtermcare_id.shape, len(np.unique(longtermcare_id))

sample: L230032066


((100000,), 97128)

In [5]:
# l번호의 중복 제거
# 중복된 ID 찾기
unique_ids, counts = np.unique(longtermcare_id, return_counts=True)
duplicates = unique_ids[counts > 1]

# ID별로 인덱스 저장
id_to_indices = defaultdict(list)
for idx, id_value in enumerate(longtermcare_id):
    id_to_indices[id_value].append(idx)

# 중복된 ID 처리
for dup_id in duplicates:
    indices = id_to_indices[dup_id]
    # 첫 번째 ID는 그대로 두고, 나머지 중복된 ID 수정
    for count, idx in enumerate(indices[1:], start=1):
        new_middle = str(count).zfill(2)  # '01', '02', '03' 등으로 변경
        new_id = dup_id[:3] + new_middle + dup_id[5:]
        # 새로운 ID가 기존 ID와 중복되지 않도록 확인
        while new_id in longtermcare_id:
            count += 1
            new_middle = str(count).zfill(2)
            new_id = dup_id[:3] + new_middle + dup_id[5:]
        # 수정된 ID를 배열에 업데이트
        longtermcare_id[idx] = new_id
        # ID 인덱스 매핑 업데이트
        id_to_indices[new_id] = [idx]

# 최종 고유 ID의 수 출력
print(f"최종 고유 ID 수: {len(np.unique(longtermcare_id))}")

최종 고유 ID 수: 100000


### 이름 생성

In [6]:
# 초성, 중성, 종성 인덱스 리스트
#초성 = ㄱ  ㄴ ㄷ ㄹ ㅁ ㅂ  ㅅ  ㅇ ㅈ  ㅊ  ㅌ ㅍ ㅎ
#중성 = ㅏ ㅐ ㅒ ㅓ  ㅕ  ㅗ ㅘ   ㅛ ㅜ    ㅠ ㅡ ㅢ ㅣ
#종성 = ㄱ  ㄴ  ㄷ ㄹ   ㅁ ㅂ  ㅅ  ㅇ ㅈ 
choseong_list = [0, 2, 3, 5, 6, 7, 9, 11, 12, 14, 16, 17, 18]
jungseong_list = [0, 1, 3, 4, 6, 8, 9, 12, 13, 17, 18, 19, 20]
jongseong_list = [0, 1, 4, 7, 8, 16, 17, 19, 21, 22]  # 받침 없는 경우 0 포함
jongseong_weights = [5 if jongseong == 0 else 1 for jongseong in jongseong_list]

# 유니코드 한글 범위에서 종성을 분리하여 쌍자음 받침을 제외하는 함수
def generate_random_korean_char():

    choseong = random.choice(choseong_list)
    jungseong = random.choice(jungseong_list)
    jongseong = random.choices(jongseong_list, weights=jongseong_weights, k=1)[0]
    
    # 유니코드 한글 음절 계산
    syllable_code = 0xAC00 + (choseong * 21 * 28) + (jungseong * 28) + jongseong
    syllable_char = chr(syllable_code)
    return syllable_char
        
#랜덤이름 만들기
def generate_random_korean_name():
    # 성만들기
    last_name = fake_ko.last_name()

    # 유니코드 한글 범위 (가~힣)
    first_char = generate_random_korean_char()  # 초성부터 종성까지 한 글자
    second_char = generate_random_korean_char()  # 두 번째 글자
    # 성과 이름 결합
    full_name = last_name + first_char + second_char

    return full_name

In [7]:
# 이름 생성
print(f'sample: {generate_random_korean_name()}')
name = np.array([generate_random_korean_name() for _ in range(num_samples)])
name.shape, len(np.unique(name))

sample: 오이섀


((100000,), 99365)

### 생년월일 생성

In [8]:
# 생년월일
def generate_birthdates(min_age, max_age, std_dev, reference_date=datetime(2024, 8, 31)):
    """
    지정된 나이 범위와 표준편차를 기반으로 정규분포된 나이를 생성하고,
    해당 나이에 맞는 생년월일 리스트를 반환하는 함수입니다.

    인자:
    - min_age (int): 최소 나이
    - max_age (int): 최대 나이
    - std_dev (float): 나이의 표준편차
    - reference_date (datetime): 기준 날짜 (기본값: 2024년 8월 31일)

    반환값:
    - birthdates (list): 생년월일 문자열의 리스트 (형식: 'YYYY-MM-DD')
    """
    # 1. 생년월일 범위 설정
    earliest_birthdate = reference_date - relativedelta(years=max_age)
    latest_birthdate = reference_date - relativedelta(years=min_age)
    
    # 2. 전체 기간의 일 수 계산
    total_days = (latest_birthdate - earliest_birthdate).days
    
    # 3. 평균 나이 및 표준편차를 일 수로 변환
    mean_age = int((min_age + max_age) / 2)  # 정수형으로 변환
    mean_birthdate = reference_date - relativedelta(years=mean_age)
    mean_day_offset = (mean_birthdate - earliest_birthdate).days
    std_dev_days = std_dev * 365.25  # 표준편차를 일 수로 변환

    # 4. 정규분포를 따르는 일 수 생성
    day_offsets = np.random.normal(loc=mean_day_offset, scale=std_dev_days, size=1)
    day_offsets = np.clip(day_offsets, 0, total_days).astype(int)
    
    # 5. 생년월일 계산
    birthdates = [earliest_birthdate + timedelta(days=int(offset)) for offset in day_offsets]
    birthdates_str = [date.strftime('%Y-%m-%d') for date in birthdates]
    
    return birthdates_str


In [9]:
# 생년월일 생성
print(f'sample: {generate_birthdates(65, 100, 6, reference_date=datetime(2024, 8, 31))}')
birth_day = np.array([generate_birthdates(65, 100, 6, reference_date=datetime(2024, 8, 31)) for _ in range(num_samples)])
birth_day.shape, len(np.unique(birth_day))

sample: ['1935-01-09']


((100000, 1), 11065)

### 주소 생성

#### get count

In [12]:
# 지역별 인구 비율 딕셔너리
population_distribution = {
    '서울특별시': 18.6,
    '경기도': 24.4,
    '부산광역시': 6.5,
    '인천광역시': 5.2,
    '대구광역시': 4.7,
    '광주광역시': 2.2,
    '대전광역시': 3.0,
    '울산광역시': 2.0,
    '세종특별자치시': 0.7,
    '강원특별자치도': 3.0,
    '충청북도': 3.2,
    '충청남도': 4.4,
    '전북특별자치도': 3.5,
    '전라남도': 3.3,
    '경상북도': 5.0,
    '경상남도': 7.4,
    '제주특별자치도': 1.7
}
# 지역 비율을 반영한 지역 선택 함수
def select_region_by_population():
    regions = list(population_distribution.keys())  # 지역 리스트
    weights = list(population_distribution.values())  # 각 지역에 대한 비율 리스트

    # random.choices를 사용하여 비율에 따라 지역을 선택
    selected_region = random.choices(regions, weights=weights, k=1)[0]
    
    return selected_region

In [13]:
# 소재지 생성
print(f'sample: {select_region_by_population()}')
region = np.array([select_region_by_population() for _ in range(num_samples)])
region.shape, len(np.unique(region))

sample: 대전광역시


((100000,), 17)

In [52]:
unique, counts = np.unique(region, return_counts=True)

region_cnt = dict(zip(unique, counts))
region_cnt

{'강원특별자치도': 3037,
 '경기도': 24777,
 '경상남도': 7493,
 '경상북도': 5075,
 '광주광역시': 2249,
 '대구광역시': 4716,
 '대전광역시': 2987,
 '부산광역시': 6607,
 '서울특별시': 18837,
 '세종특별자치시': 704,
 '울산광역시': 1942,
 '인천광역시': 5303,
 '전라남도': 3323,
 '전북특별자치도': 3599,
 '제주특별자치도': 1710,
 '충청남도': 4399,
 '충청북도': 3242}

#### get address index

In [ ]:
## 주소 index 생성
address = pd.read_csv('./gps_data/merged_output.csv')
address

,SIGUNGU_CD,ENT_NO,LAWDONG_CD,SIDO_NM,SIGUNGU_NM,EUPMYUNDONG_NM,DORO_CD,DORO_NM,UNDER_YN,BUILDING_NO,BUILDING_SUB_NO,BUILDING_NM,POST_NO,BUILDING_KIND,BUILDING_GROUP_YN,HAENGJUNGDONG_NM,X_LONG,Y_LAN,GPS_LON_X,GPS_LAT_Y
0,51110,27185,5111010100,강원특별자치도,춘천시,봉의동,511104454226,모수물길,0,11,0,NaN,24269,주택,0,소양동,1.020096e+06,1.987311e+06,127.742543,37.956362
1,51110,27184,5111010100,강원특별자치도,춘천시,봉의동,511104454226,모수물길,0,13,0,NaN,24269,주택,0,소양동,1.020100e+06,1.987323e+06,127.767055,37.967900
2,51110,19704,5111010100,강원특별자치도,춘천시,봉의동,511104454226,모수물길,0,32,0,봉의현대빌라,24249,주택,0,소양동,1.020090e+06,1.987488e+06,127.630631,37.875876
3,51110,19705,5111010100,강원특별자치도,춘천시,봉의동,511104454226,모수물길,0,34,0,봉의연립,24249,주택,1,소양동,1.020078e+06,1.987494e+06,127.826483,37.895621
4,51110,42303,5111010100,강원특별자치도,춘천시,봉의동,511104454226,모수물길,0,38,0,NaN,24249,주택,0,소양동,1.020058e+06,1.987531e+06,127.755266,37.930871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3696581,43800,13719,4380036000,충청북도,단양군,단성면,438004544559,월악중방길,0,106,0,NaN,27025,주택,0,단성면,1.072306e+06,1.882410e+06,128.361958,37.034431
3696582,43800,13718,4380036000,충청북도,단양군,단성면,438004544559,월악중방길,0,108,0,NaN,27025,주택,0,단성면,1.072292e+06,1.882400e+06,128.287641,36.941656
3696583,43800,13715,4380036000,충청북도,단양군,단성면,438004544559,월악중방길,0,112,0,NaN,27025,주택,0,단성면,1.072261e+06,1.882381e+06,128.213329,36.880242
3696584,43800,13716,4380036000,충청북도,단양군,단성면,438004544559,월악중방길,0,112,1,NaN,27025,주택,0,단성면,1.072260e+06,1.882398e+06,128.217750,36.863009


In [58]:
# data 준비
## address에서 sido_nm과 인덱스만 가져오기
address_and_index = np.column_stack((address['SIDO_NM'].to_numpy(), address.index.to_numpy()))
address_and_index, len(address_and_index), address.SIDO_NM.unique()  ## good

(array([['강원특별자치도', 0],
        ['강원특별자치도', 1],
        ['강원특별자치도', 2],
        ...,
        ['충청북도', 3696583],
        ['충청북도', 3696584],
        ['충청북도', 3696585]], dtype=object),
 3696586,
 array(['강원특별자치도', '경기도', '경상남도', ..., '제주특별자치도', '충청남도', '충청북도'],
       dtype=object))

In [56]:
# 각 SIDO_NM별로 고유한 인덱스 추출
sampled_indices = []

for region, count in region_cnt.items():

    # 해당 region에 속하는 인덱스들을 추출
    region_indices = address_and_index[address_and_index[:, 0] == region][:, 1].astype(int)
    
    # count에 해당하는 무작위 샘플링
    sampled = np.random.choice(region_indices, size=count, replace=False)
    
    # 결과를 리스트에 추가
    sampled_indices.extend(sampled)

'샘플갯수', len(sampled_indices),'고유값', len(np.unique(sampled_indices))

('샘플갯수', 100000, '고유값', 100000)

#### [code rip] 생성된 주소별로 주택주소db 랜덤 인덱싱

In [37]:
# #### 속도가 너무 느려서 폐기
# # region index 선택
# # 중복을 제거하고

# # 인덱스를 저장할 빈 배열
# result_indices = np.array([])
# total_region = len(region)

# # 수급자 시도 정보에 대해 주소정보와 시도를 매칭하여 랜덤 인덱스 선택

# for i, prs_region in enumerate(region):
#     print(f'Processing region {i+1:6<}/{total_region} {(i+1)/(total_region)*100:.2f}%', end='\r')

#     # 수급자 시와 일치하는 주소 목록 필터링
#     matching_addresses = address_and_index[address_and_index[:, 0] == prs_region]

#     # 사용하지 않은 인덱스 남기기
#     all_indices = matching_addresses[:, 1].astype(int)  # 인덱스 부분을 가져옴
#     available_indices = np.setdiff1d(all_indices, np.array(list(result_indices)))  # 사용된 인덱스 제외
    
#         # 일치하는 주소가 있으면 랜덤 인덱스 선택
#     if available_indices.size > 0:
#         random_index = np.random.choice(available_indices)  # 랜덤 선택 <-numpy 선택 방식
#             #try1: random_index = random.choice(matching_addresses['Index'].tolist()) <-- 내장 리스트 방식
#             #try2: random_index = matching_addresses.sample(n=1).index[0]  <-- pd sample 선택 방식
#             # 둘다 속도 너무 느림...
#         result_indices = np.append(result_indices, random_index)  # 선택한 인덱스를 넘파이 배열에 추가

#     else:
#         print('address is full!!')
#         break
# result_indices, result_indices.shape 

KeyboardInterrupt: 

### 등급 생성

In [70]:
# 각 등급별로 랜덤 샘플링
grade_list = np.arange(1, 6) ## 1~5
grade_ratio = [4.2, 7.8, 24.0, 40.9, 10.3]
grade_ratio = np.array(grade_ratio) / 100 * num_samples
grade_ratio = grade_ratio / grade_ratio.sum()
grade = np.random.choice(grade_list, size=num_samples, p=grade_ratio)
'result:', grade, grade.shape, np.unique(grade)

('result:', array([4, 1, 4, ..., 4, 4, 5]), (100000,), array([1, 2, 3, 4, 5]))